In [13]:
import os

import pymongo
from bson.json_util import dumps, loads
from bson.objectid import ObjectId

from splinter import Browser
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0
from selenium.common.exceptions import TimeoutException

import pandas as pd

import re
from pprint import pprint
from datetime import datetime as dt
import time
import isodate

import sys
import pandas as pd
import json
import os
from pprint import pprint
import random

In [2]:
os.getcwd()

'/home/shortpoet/Repos/language'

In [3]:
# executable_path = {'executable_path': '/usr/bin/chromedriver'}
# browser = Browser('chrome')#, **executable_path)
driver = webdriver.Chrome(executable_path='/usr/bin/chromedriver')

In [4]:
url = 'https://www.seriouseats.com/sitemap'
driver.get(url)

In [5]:
html = driver.page_source
se_sm_soup = bs(html, 'html.parser')

In [6]:
url_dict = {}
topic_anchors = se_sm_soup.find_all('a', href=re.compile('topics'))

In [7]:
url_dict['ingredient'] = []
url_dict['method'] = []
url_dict['meal'] = []
url_dict['cuisine'] = []

for i, anchor in enumerate(topic_anchors):
#     print(f"{i}: {anchor.text.strip()}")
#     print(anchor['href'])
    link = anchor['href']
    if 'ingredient' in link:
        this_dict = {'topic': anchor.text.strip(), 'ingredient_url(s)': [link]}
        url_dict['ingredient'].append(this_dict)
    if 'method' in link:
        this_dict = {'topic': anchor.text.strip(), 'method_url(s)': [link]}
        url_dict['method'].append(this_dict)
    if 'meal' in link:
        this_dict = {'topic': anchor.text.strip(), 'meal_url(s)': [link]}
        url_dict['meal'].append(this_dict)
    if 'cuisine' in link:
        this_dict = {'topic': anchor.text.strip(), 'cuisine_url(s)': [link]}
        url_dict['cuisine'].append(this_dict)

In [8]:
url_dict['cuisine'] = url_dict['cuisine'][1:]
url_dict['ingredient'] = url_dict['ingredient'][0:2] + url_dict['ingredient'][3:]
url_dict['method'] = url_dict['method'][0:2] + url_dict['method'][3:]
pprint(url_dict)

{'cuisine': [{'cuisine_url(s)': ['https://www.seriouseats.com/recipes/topics/cuisine/african'],
              'topic': 'African Recipes'},
             {'cuisine_url(s)': ['https://www.seriouseats.com/recipes/topics/cuisine/american'],
              'topic': 'American Recipes'},
             {'cuisine_url(s)': ['https://www.seriouseats.com/recipes/topics/cuisine/asian'],
              'topic': 'Asian Recipes'},
             {'cuisine_url(s)': ['https://www.seriouseats.com/recipes/topics/cuisine/caribbean'],
              'topic': 'Caribbean Recipes'},
             {'cuisine_url(s)': ['https://www.seriouseats.com/recipes/topics/cuisine/asian/chinese'],
              'topic': 'Chinese Recipes'},
             {'cuisine_url(s)': ['https://www.seriouseats.com/recipes/topics/cuisine/asian/filipino'],
              'topic': 'Filipino Recipes'},
             {'cuisine_url(s)': ['https://www.seriouseats.com/recipes/topics/cuisine/french'],
              'topic': 'French Recipes'},
             

In [9]:
def href_filt(href):
    return re.compile('recipe').search(href) and re.compile('https').search(href) and not re.compile('topics').search(href)

In [10]:
test = {'cuisine': url_dict['cuisine']}
for topic, _list in url_dict.items():
    for _dict in _list:
        _dict.setdefault('recipes', [])
        string = topic + '_url(s)'
        print(f"string: {string}")
        for i, u in enumerate(_dict[string]):
            cfTopic = re.sub(r'\s', '_', _dict['topic'])
            cfTopic = re.sub(r'/', '_', cfTopic)
            print(cfTopic)
            cacheFilePath = f"/home/shortpoet/Sources/serious_eats_html_caches/recipe_urls/{cfTopic}_{i}.txt"
            print(f"cfp: {cacheFilePath}")
            if os.path.isfile(cacheFilePath):
                with open(cacheFilePath, encoding='utf-8') as cacheFile:
                    html = cacheFile.read()
                    se_recipe_soup = bs(html, 'html.parser')
                    content = se_recipe_soup.find('div', class_='content-main')
                    recipe_anchors = content.find_all('a', href=href_filt)
                    for anchor in recipe_anchors[::2]:
                        this_dict = {'recipe_title': anchor.text, 'recipe_url': anchor['href']}
                        _dict['recipes'].append(this_dict)
            else:
                url = u
                print(f"url: {url}")
                time.sleep(random.uniform(0,3))
                driver.get(url)
                html = driver.page_source
                with open(cacheFilePath, "w", encoding='utf-8') as cacheFile:
                    cacheFile.write(html)        

                se_recipe_soup = bs(html, 'html.parser')
                content = se_recipe_soup.find('div', class_='content-main')
                recipe_anchors = content.find_all('a', href=href_filt)
                recipe_anchors = recipe_anchors[2:]
                for anchor in recipe_anchors[::2]:
                    this_dict = {'recipe_title': anchor.text, 'recipe_url': anchor['href']}
                    _dict['recipes'].append(this_dict)
                if se_recipe_soup.find('a', text=(re.compile('Next'))):
                    time.sleep(random.uniform(0,5))
                    url = se_recipe_soup.find('a', text=(re.compile('Next')))['href']
                    _dict[string].append(url)
                    driver.get(url)
                    html = driver.page_source
                    se_recipe_soup = bs(html, 'html.parser')
                    content = se_recipe_soup.find('div', class_='content-main')
                    recipe_anchors = content.find_all('a', href=href_filt)
                    for anchor in recipe_anchors[::2]:
                        this_dict = {'recipe_title': anchor.text, 'recipe_url': anchor['href']}
                        _dict['recipes'].append(this_dict)

string: ingredient_url(s)
Chicken
cfp: /home/shortpoet/Sources/serious_eats_html_caches/recipe_urls/Chicken_0.txt
string: ingredient_url(s)
Pasta
cfp: /home/shortpoet/Sources/serious_eats_html_caches/recipe_urls/Pasta_0.txt
string: ingredient_url(s)
Cheese
cfp: /home/shortpoet/Sources/serious_eats_html_caches/recipe_urls/Cheese_0.txt
string: ingredient_url(s)
Eggs
cfp: /home/shortpoet/Sources/serious_eats_html_caches/recipe_urls/Eggs_0.txt
string: ingredient_url(s)
Deviled_Egg_Recipes
cfp: /home/shortpoet/Sources/serious_eats_html_caches/recipe_urls/Deviled_Egg_Recipes_0.txt
string: ingredient_url(s)
Egg_Salad_Recipes
cfp: /home/shortpoet/Sources/serious_eats_html_caches/recipe_urls/Egg_Salad_Recipes_0.txt
string: ingredient_url(s)
Eggs_Benedict_Recipes
cfp: /home/shortpoet/Sources/serious_eats_html_caches/recipe_urls/Eggs_Benedict_Recipes_0.txt
string: ingredient_url(s)
Fried_Egg_Recipes
cfp: /home/shortpoet/Sources/serious_eats_html_caches/recipe_urls/Fried_Egg_Recipes_0.txt
string: 

string: method_url(s)
Deep_Fried_Recipes
cfp: /home/shortpoet/Sources/serious_eats_html_caches/recipe_urls/Deep_Fried_Recipes_0.txt
string: method_url(s)
Grilling,_Smoking,_and_Barbecue_Recipes
cfp: /home/shortpoet/Sources/serious_eats_html_caches/recipe_urls/Grilling,_Smoking,_and_Barbecue_Recipes_0.txt
string: method_url(s)
No-Cook_Recipes
cfp: /home/shortpoet/Sources/serious_eats_html_caches/recipe_urls/No-Cook_Recipes_0.txt
string: method_url(s)
Poaching_Recipes
cfp: /home/shortpoet/Sources/serious_eats_html_caches/recipe_urls/Poaching_Recipes_0.txt
string: method_url(s)
Pressure_Cooker_Recipes
cfp: /home/shortpoet/Sources/serious_eats_html_caches/recipe_urls/Pressure_Cooker_Recipes_0.txt
string: method_url(s)
Roasting_Recipes
cfp: /home/shortpoet/Sources/serious_eats_html_caches/recipe_urls/Roasting_Recipes_0.txt
string: method_url(s)
Slow_Cooker_Recipes
cfp: /home/shortpoet/Sources/serious_eats_html_caches/recipe_urls/Slow_Cooker_Recipes_0.txt
string: method_url(s)
Sous_Vide_Reci

In [14]:
# pprint(se_recipe_links_dict)
# se_recipe_links_dict.keys()
sys.getsizeof(url_dict)

240

In [15]:
cacheFilePath = f"/home/shortpoet/Sources/serious_eats_html_caches/recipe_urls/"
filess = []
for root, dirs, files in os.walk(cacheFilePath):

    for file in files:
        filess.append(file)


In [16]:
for i, file in enumerate(filess):
    print(f"processing:{i} of {len(filess)}: {file}")
    for topic, _list in url_dict.items():
        for _dict in _list:
            _dict.setdefault('recipes', [])
            cfTopic = re.sub(r'\s', '_', _dict['topic'])
            cfTopic = re.sub(r'/', '_', cfTopic)
            if cfTopic in file:
                cacheFilePath = f"/home/shortpoet/Sources/serious_eats_html_caches/recipe_urls/{file}"
                with open(cacheFilePath, encoding='utf-8') as cacheFile:
                    html = cacheFile.read()
                    se_recipe_soup = bs(html, 'html.parser')
                    content = se_recipe_soup.find('div', class_='content-main')
                    recipe_anchors = content.find_all('a', href=href_filt)
                    recipe_anchors = recipe_anchors[2:]
                    for anchor in recipe_anchors[::2]:
                        this_dict = {'recipe_title': anchor.text, 'recipe_url': anchor['href']}
                        _dict['recipes'].append(this_dict)

processing:0 of 1235: Vegetables_64.txt
processing:1 of 1235: Desserts_69.txt
processing:2 of 1235: Champagne_Cocktails_0.txt
processing:3 of 1235: Cheese_7.txt
processing:4 of 1235: Omelette_Recipes_1.txt
processing:5 of 1235: Shrimp_Recipes_8.txt
processing:6 of 1235: Potato_Recipes_8.txt
processing:7 of 1235: Tomato_Recipes_7.txt
processing:8 of 1235: Brandy_and_Cognac_Drinks_3.txt
processing:9 of 1235: Vegetables_38.txt
processing:10 of 1235: Brandy_and_Cognac_Drinks_1.txt
processing:11 of 1235: Mexican_Recipes_10.txt
processing:12 of 1235: Cauliflower_Recipes_3.txt
processing:13 of 1235: Mexican_Recipes_12.txt
processing:14 of 1235: Potato_Recipes_10.txt
processing:15 of 1235: Chicken_Recipes_2.txt
processing:16 of 1235: Liqueur_and_Fortified-Wine_Cocktails_1.txt
processing:17 of 1235: Other_Grain_Recipes_0.txt
processing:18 of 1235: Desserts_30.txt
processing:19 of 1235: Desserts_35.txt
processing:20 of 1235: Asian_Recipes_20.txt
processing:21 of 1235: Indian_Recipes_6.txt
proces

processing:189 of 1235: Vegetables_33.txt
processing:190 of 1235: Desserts_93.txt
processing:191 of 1235: Seafood_8.txt
processing:192 of 1235: Desserts_82.txt
processing:193 of 1235: Cheese_16.txt
processing:194 of 1235: Stir-Fry_Recipes_1.txt
processing:195 of 1235: Vegetables_47.txt
processing:196 of 1235: Pork_Recipes_18.txt
processing:197 of 1235: Lamb_Recipes_3.txt
processing:198 of 1235: Cocktails_44.txt
processing:199 of 1235: Cabbage_Recipes_0.txt
processing:200 of 1235: Meats_and_Poultry_1.txt
processing:201 of 1235: Italian_Recipes_3.txt
processing:202 of 1235: Desserts_38.txt
processing:203 of 1235: Meats_and_Poultry_61.txt
processing:204 of 1235: Meats_and_Poultry_70.txt
processing:205 of 1235: Spanish_Recipes_2.txt
processing:206 of 1235: Lasagna_Recipes_0.txt
processing:207 of 1235: Meats_and_Poultry_50.txt
processing:208 of 1235: Desserts_100.txt
processing:209 of 1235: Grains_19.txt
processing:210 of 1235: Desserts_85.txt
processing:211 of 1235: Grilled_Chicken_Recipes

processing:378 of 1235: By_Method_6.txt
processing:379 of 1235: Seafood_3.txt
processing:380 of 1235: Lamb_Recipes_6.txt
processing:381 of 1235: Cheese_5.txt
processing:382 of 1235: Asian_Recipes_28.txt
processing:383 of 1235: Latin_American_Recipes_2.txt
processing:384 of 1235: Cocktails_10.txt
processing:385 of 1235: Rice_Recipes_6.txt
processing:386 of 1235: Lobster_and_Crab_Recipes_0.txt
processing:387 of 1235: Pork_Recipes_22.txt
processing:388 of 1235: Grains_16.txt
processing:389 of 1235: Salad_13.txt
processing:390 of 1235: Gin_Cocktails_5.txt
processing:391 of 1235: Pasta_3.txt
processing:392 of 1235: Chicken_Recipes_22.txt
processing:393 of 1235: Desserts_27.txt
processing:394 of 1235: Desserts_21.txt
processing:395 of 1235: Vegetables_65.txt
processing:396 of 1235: Meats_and_Poultry_36.txt
processing:397 of 1235: Pork_Recipes_0.txt
processing:398 of 1235: Pasta_19.txt
processing:399 of 1235: Chinese_Recipes_11.txt
processing:400 of 1235: Moroccan_North_African_Recipes_0.txt


processing:566 of 1235: View_All_Cocktails_11.txt
processing:567 of 1235: View_All_Cocktails_43.txt
processing:568 of 1235: Chinese_Recipes_6.txt
processing:569 of 1235: Desserts_14.txt
processing:570 of 1235: View_All_Cocktails_18.txt
processing:571 of 1235: Korean_Recipes_2.txt
processing:572 of 1235: Japanese_Recipes_1.txt
processing:573 of 1235: Cabbage_Recipes_3.txt
processing:574 of 1235: Cucumber_Recipes_0.txt
processing:575 of 1235: Mashed_Potato_Recipes_1.txt
processing:576 of 1235: Eggs_2.txt
processing:577 of 1235: French_Recipes_3.txt
processing:578 of 1235: Desserts_22.txt
processing:579 of 1235: Chicken_Recipes_6.txt
processing:580 of 1235: View_All_Cocktails_8.txt
processing:581 of 1235: Vegetables_24.txt
processing:582 of 1235: Pork_Recipes_26.txt
processing:583 of 1235: Desserts_6.txt
processing:584 of 1235: Shrimp_Recipes_0.txt
processing:585 of 1235: Vegetables_12.txt
processing:586 of 1235: Salad_8.txt
processing:587 of 1235: Meats_and_Poultry_85.txt
processing:588 

processing:753 of 1235: Cocktails_33.txt
processing:754 of 1235: Deep_Fried_Recipes_2.txt
processing:755 of 1235: Quinoa_Recipes_1.txt
processing:756 of 1235: Seafood_11.txt
processing:757 of 1235: Vodka_1.txt
processing:758 of 1235: Chinese_Recipes_1.txt
processing:759 of 1235: Desserts_74.txt
processing:760 of 1235: Chicken_Recipes_32.txt
processing:761 of 1235: Vodka_0.txt
processing:762 of 1235: View_All_Cocktails_21.txt
processing:763 of 1235: Corn_Recipes_5.txt
processing:764 of 1235: Whiskey_Cocktails_5.txt
processing:765 of 1235: Salad_26.txt
processing:766 of 1235: Braising_and_Stewing_Recipes_3.txt
processing:767 of 1235: Gin_8.txt
processing:768 of 1235: Pasta_17.txt
processing:769 of 1235: Chicken_Recipes_13.txt
processing:770 of 1235: Salad_20.txt
processing:771 of 1235: Pork_Recipes_23.txt
processing:772 of 1235: Eggplant_Recipes_3.txt
processing:773 of 1235: Corn_Recipes_3.txt
processing:774 of 1235: Italian_Recipes_11.txt
processing:775 of 1235: Cocktails_37.txt
process

processing:942 of 1235: Slow_Cooker_Recipes_1.txt
processing:943 of 1235: Cocktails_20.txt
processing:944 of 1235: Salad_30.txt
processing:945 of 1235: Meats_and_Poultry_33.txt
processing:946 of 1235: Asian_Recipes_7.txt
processing:947 of 1235: Meats_and_Poultry_14.txt
processing:948 of 1235: Salad_9.txt
processing:949 of 1235: Desserts_33.txt
processing:950 of 1235: Deep_Fried_Recipes_6.txt
processing:951 of 1235: Seafood_2.txt
processing:952 of 1235: Desserts_101.txt
processing:953 of 1235: View_All_Cocktails_13.txt
processing:954 of 1235: Grilled_Chicken_Recipes_0.txt
processing:955 of 1235: Italian_Recipes_8.txt
processing:956 of 1235: Meats_and_Poultry_72.txt
processing:957 of 1235: Other_Vegetable_Recipes_3.txt
processing:958 of 1235: Meats_and_Poultry_22.txt
processing:959 of 1235: Meats_and_Poultry_20.txt
processing:960 of 1235: Kale_Recipes_0.txt
processing:961 of 1235: Pasta_24.txt
processing:962 of 1235: Vegetables_25.txt
processing:963 of 1235: Desserts_48.txt
processing:96

processing:1127 of 1235: View_All_42.txt
processing:1128 of 1235: Baking_Recipes_6.txt
processing:1129 of 1235: Spanish_Recipes_1.txt
processing:1130 of 1235: Cocktails_14.txt
processing:1131 of 1235: Seafood_17.txt
processing:1132 of 1235: Zucchini_Recipes_1.txt
processing:1133 of 1235: Other_Seafood_Recipes_3.txt
processing:1134 of 1235: Cocktails_34.txt
processing:1135 of 1235: Desserts_36.txt
processing:1136 of 1235: Meats_and_Poultry_3.txt
processing:1137 of 1235: Seafood_30.txt
processing:1138 of 1235: Broccoli_Recipes_0.txt
processing:1139 of 1235: Italian_Recipes_10.txt
processing:1140 of 1235: Beef_Recipes_24.txt
processing:1141 of 1235: View_All_33.txt
processing:1142 of 1235: Asian_Recipes_32.txt
processing:1143 of 1235: Desserts_47.txt
processing:1144 of 1235: View_All_Cocktails_31.txt
processing:1145 of 1235: French_Recipes_11.txt
processing:1146 of 1235: Meats_and_Poultry_100.txt
processing:1147 of 1235: Vodka_Drinks_2.txt
processing:1148 of 1235: Grains_12.txt
processing

In [17]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
mongoDb = client.serious_eats_urls


In [18]:
collection = mongoDb.se_urls
collection.insert_one(url_dict)

In [ ]:
collection = mongoDb.se_urls
url_dict = collection.find_one({}, {'_id':False})

In [ ]:
collection = mongoDb.se_recipe_just_url_test
unupdated_dict = collection.find_one({}, {"recipes.updated":{'$exists': False}})
unupdated_dict

In [ ]:
for topic, topic_list in url_dict.items():
    for index, subtopic_dict in enumerate(topic_list):
        for _, _ in subtopic_dict.items():
            for subindex, recipe_dict in enumerate(subtopic_dict['recipes']):
                print(recipe_dict['recipe_title'])
                

In [ ]:
b.nested.findOne({"level1":{"$elemMatch":{"$in":['item00']}} })

In [ ]:
# grab full list of recipe urls

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
mongoDb = client.serious_eats_urls

collection = mongoDb.se_urls
url_dict = collection.find_one({}, {'_id':False})

In [19]:
# create new database with collection names spread out

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
mongoDb = client.serious_eats

for topic, topic_list in url_dict.items():
    print(f'topic: {topic}')
    for index, subtopic_dict in enumerate(topic_list):
        collection = mongoDb[subtopic_dict['topic']]
        collection.create_index('recipe_url', unique=True)
        for _, _ in subtopic_dict.items():
            for subindex, recipe_dict in enumerate(subtopic_dict['recipes']):
                url = recipe['recipe_url']
                print(f"{topic}: {i} of {len(topic_dict['recipes'])} => {url}")
                title = recipe['recipe_title']
                this_recipe = {
                    'topic': subtopic_dict['topic'], 'source': 'serious_eats', 
                    'recipe_title': recipe['recipe_title'], 'recipe_url': recipe['recipe_url']
                }
                try:
                    collection.insert_one(this_recipe)
                except pymongo.errors.DuplicateKeyError:
                    continue

topic: ingredient


NameError: name 'recipe' is not defined

In [ ]:
# make list from collection names minus full url list collection

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
mongoDb = client.serious_eats_test2

collection_names = mongoDb.list_collection_names()
coll_names = []
for i, coll in enumerate(collection_names):
    print(i, coll)
    coll_names.append(coll)

coll_names = coll_names[:105] + coll_names[106:107]
coll_names

In [ ]:
for i, coll in enumerate(coll_names):
    print(i, coll)

In [ ]:
# set updated timestamp to url collection

for topic, topic_list in url_dict.items():
    for index, subtopic_dict in enumerate(topic_list):
        for _, _ in subtopic_dict.items():
            for subindex, recipe_dict in enumerate(subtopic_dict['recipes']):
                this_url = recipe_dict['recipe_url']
                mongoDb = client.serious_eats_test
                collection = mongoDb[subtopic_dict['topic']]
                if collection.find_one({'recipe_url':this_url}):
#                     collection = mongoDb.se_recipe_just_url
                    this_recipe = {
                        'topic': subtopic_dict['topic'], 'source': 'serious_eats', 
                        'recipe_title': recipe_dict['recipe_title'],
                        'recipe_url': recipe_dict['recipe_url'],
                        'updated': dt.now()
                    }
                    this_query = f"{subindex}"
                    print(f"processing: {subtopic_dict['topic']} => {this_query}")
                    mongoDb = client.serious_eats_test2
                    collection = mongoDb[subtopic_dict['topic']]
                    collection.update_one({'recipe_url': this_url}, {"$set": this_recipe}, upsert=True)


In [ ]:
# create dictionary of unupdated ids

mongoDb = client.serious_eats_test2
unupdated = {}
for coll in coll_names:
    collection = mongoDb[coll]
    unupdated[coll] = []
    r = collection.find({"updated":{'$exists': False}}, {})
    for i in r:
        unupdated[coll].append(i['_id'])


In [ ]:
unupdated

In [ ]:
# create dictionary of unupdated recipe urls from ids


unupdated_url_dict = {}
for coll, obj_ids in unupdated.items():
    collection = mongoDb[coll]
    unupdated_url_dict[coll] = []
    for _id in obj_ids:
        r = collection.find_one({'_id': ObjectId(_id)})
        print(f"processing:{coll}{_id}")
        topic = r['topic']
        url = r['recipe_url']
        title = r['recipe_title']
        this_dict = {'topic': topic, 'recipe_url': url, 'recipe_title': title}
        unupdated_url_dict[coll].append(this_dict)


In [ ]:
for coll, recipe_list in unupdated_url_dict.items():
    collection = mongoDb[coll]
    for recipe_dict in recipe_list:
        recipe_dict.setdefault('recipes', [])
        for i, u in enumerate(recipe_dict):
            cfTopic = re.sub(r'\s', '_', recipe_dict['topic'])
            cfTopic = re.sub(r'/', '_', cfTopic)
            print(cfTopic)
            cacheFilePath = f"/home/shortpoet/Sources/serious_eats_html_caches/recipe_urls/{cfTopic}_{i}.txt"
            print(f"cfp: {cacheFilePath}")
            if os.path.isfile(cacheFilePath):
                with open(cacheFilePath, encoding='utf-8') as cacheFile:
                    html = cacheFile.read()
                    se_recipe_soup = bs(html, 'html.parser')
                    content = se_recipe_soup.find('div', class_='content-main')
                    recipe_anchors = content.find_all('a', href=href_filt)
                    for anchor in recipe_anchors[::2]:
                        this_dict = {'recipe_title': anchor.text, 'recipe_url': anchor['href']}
                        recipe_dict['recipes'].append(this_dict)
                        collection.update_one({'recipe_url':recipe_dict['recipe_url']}, {'$set':{'recipes':anchor['href']}}, upsert=True)

In [21]:
## use if url_dict is created from scratch
mongoDb = client.serious_eats
for topic, topic_list in url_dict.items():
    print(f'topic: {topic}')
    for topic_dict in topic_list:
        collection = mongoDb[topic_dict['topic']]
        collection.create_index('recipe_url', unique=True)
        for i, recipe in enumerate(topic_dict['recipes']):
            url = recipe['recipe_url']
            print(f"{topic}: {i} of {len(topic_dict['recipes'])} => {url}")
            cfTitle = re.sub(r"\s|/|\.|,|'|-|\?|\||<|>|\\|\*|:", '_', recipe['recipe_title'])
            cfTitle = re.sub(r'"', '', cfTitle)
            cfTitle = re.sub(r'__', '_', cfTitle)
            if cfTitle[0:1] == "_":
                cfTitle = cfTitle[1:]
            if cfTitle[-1:] == "_":
                cfTitle = cfTitle[:-1]
            cfTitle = cfTitle[:32]
            print(f"cfTitle: {cfTitle}")
            cacheFilePath = f"/home/shortpoet/Sources/serious_eats_html_caches/{cfTitle}.txt"
            print(f"cfp: {cacheFilePath}")
            
            if os.path.isfile(cacheFilePath):
                with open(cacheFilePath, encoding='utf-8') as cacheFile:
                    html = cacheFile.read()
            else:
                driver.get(url)
                html = driver.page_source          
                with open(cacheFilePath, 'w', encoding='utf-8') as cacheFile:
                    cacheFile.write(html)
            se_soup = bs(html, 'html.parser')
            script_divs = se_soup.find_all('script', {'type': 'application/ld+json'})
#             recipe['category_trees'] = []
#             recipe['recipe'] = []
            category_trees = []
            for index, div in enumerate(script_divs):
                try:
                    j = json.loads(div.text, strict=False)

                    recipe.setdefault('category_trees', [])
                    recipe.setdefault('recipe', {})

                    if j['@type'] == 'BreadcrumbList':
                        category_tree = []
                        for item in j['itemListElement']:
                            category_tree.append(item['item']['name'])
    #                     recipe['category_trees'].append(category_tree)
                        category_trees.append(category_tree)

                    if j['@type'] == 'Recipe':
                        try:
                            rating_count = j['aggregateRating']['ratingCount']
                        except:
                            rating_count = False
                        try:
                            rating_value = j['aggregateRating']['ratingValue']
                        except:
                            rating_value = False
                        try:
                            author = j['author']['name']
                        except:
                            author = False
                        try:
                            job_title = j['author']['jobTitle']
                        except:
                            job_title = False
                        try:
                            description = j['description']
                        except:
                            description = False
                        try:
                            keywords = j['keywords']
                        except:
                            keywords = False
                        try: 
                            name = j['name']
                        except:
                            name = False
                        try:
                            categories = j['recipeCategory']
                        except:
                            categories = False
                        try:
                            cuisine = j['recipeCuisine']
                        except:
                            cuisine = False
                        try:
                            ingredients = j['recipeIngredient']
                        except:
                            ingredients = False
                        try:   
                            steps = {}
                            for i, step in enumerate(j['recipeInstructions']):
                                this = 'step_' + str(i+1)
                                steps[this] = step['text']
                        except:
                            steps = False
                        try:
                            recipe_yield = j['recipeYield']
                        except:
                            recipe_yield = False
                        try:
                            time = str(isodate.parse_duration(j['totalTime']).total_seconds()/60)
                        except:
                            time = False
    #                     recipe['recipe'] = {
                        json_recipe = {
                            'name': name, 'rating_count': rating_count, 'rating_value': rating_value, 'author': author,
                            'job_title': job_title, 'description': description, 'keywords': keywords, 'categories': categories,
                            'cuisine': cuisine, 'ingredients': ingredients, 'steps': steps, 'recipe_yield': recipe_yield, 'time':time
                        }
                        print(json_recipe)
                except:
                    print(f"json error on:{url} - script_div{index}")
                    continue
            this_recipe = {
                'topic': topic_dict['topic'], 'source': 'serious_eats', 
                'recipe_title': recipe['recipe_title'], 'recipe_url': recipe['recipe_url'], 
#                 'category_trees': recipe['category_trees'], 'recipe': recipe['recipe'], 
                'category_trees': recipe['category_trees'], 'recipe': json_recipe, 
                'uploaded': dt.now()
            }
            try:
                collection.insert_one(this_recipe)
            except pymongo.errors.DuplicateKeyError:
                continue
#             collection.update_one({}, {'$set': collection.this_recipe}, upsert=True)


topic: ingredient
ingredient: 0 of 1110 => https://www.seriouseats.com/recipes
cfTitle: _____Recipes
cfp: /home/shortpoet/Sources/serious_eats_html_caches/_____Recipes.txt


NameError: name 'json_recipe' is not defined

In [ ]:
cfTitle = re.sub(r"\s|/|\.|,|'|-", '_', 'C:\\Users\\soria\\Documents\\resources\\caches\\serious_eats_html_caches\\Weeknight_Turkey_"Bolognese".txt')
cfTitle = re.sub(r'"', '', cfTitle)
cfTitle = re.sub(r'__', '_', cfTitle)
cfTitle

In [ ]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
mongoDb = client.serious_eats_test_2


In [ ]:
# se_recipe_links_dict.pop('_id')
for topic, _list in url_dict.items():
    collection = mongoDb[topic]
    collection.insert_many(_list)

In [ ]:
## use if pulled from mongo

for topic, topic_dict in url_dict.items():
    print(f'topic: {topic}')
    for i, recipe in enumerate(topic_dict['recipes']):
        url = recipe['recipe_url']
        print(f"{topic}: {i} of {len(topic_dict['recipes'])} => {url}")
        driver.get(url)
        html = driver.page_source
        se_soup = bs(html, 'html.parser')
        script_divs = se_soup.find_all('script', {'type': 'application/ld+json'})
        for div in script_divs:
            j = json.loads(div.text, strict=False)

            recipe.setdefault('category_trees', [])
            recipe.setdefault('recipe', {})

            if j['@type'] == 'BreadcrumbList':
                category_tree = []
                for item in j['itemListElement']:
                    category_tree.append(item['item']['name'])
                recipe['category_trees'].append(category_tree)
            if j['@type'] == 'Recipe':
                try:
                    rating_count = j['aggregateRating']['ratingCount']
                except:
                    rating_count = False
                try:
                    rating_value = j['aggregateRating']['ratingValue']
                except:
                    rating_value = False
                try:
                    author = j['author']['name']
                except:
                    author = False
                try:
                    job_title = j['author']['jobTitle']
                except:
                    job_title = False
                try:
                    description = j['description']
                except:
                    description = False
                try:
                    keywords = j['keywords']
                except:
                    keywords = False
                try: 
                    name = j['name']
                except:
                    name = False
                try:
                    categories = j['recipeCategory']
                except:
                    categories = False
                try:
                    cuisine = j['recipeCuisine']
                except:
                    cuisine = False
                try:
                    ingredients = j['recipeIngredient']
                except:
                    ingredients = False
                try:   
                    steps = {}
                    for i, step in enumerate(j['recipeInstructions']):
                        this = 'step_' + str(i+1)
                        steps[this] = step['text']
                except:
                    steps = False
                try:
                    recipe_yield = j['recipeYield']
                except:
                    recipe_yield = False
                try:
                    time = str(isodate.parse_duration(j['totalTime']).total_seconds()/60)
                except:
                    time = False
                recipe['recipe'] = {
                    'name': name, 'rating_count': rating_count, 'rating_value': rating_value, 'author': author,
                    'job_title': job_title, 'description': description, 'keywords': keywords, 'categories': categories,
                    'cuisine': cuisine, 'ingredients': ingredients, 'steps': steps, 'recipe_yield': recipe_yield, 'time':time
                }



In [ ]:
## use for unupdated dict

mongoDb = client.serious_eats_test

for coll, recipe_list in unupdated_url_dict.items():
    
    collection = mongoDb[coll]
    collection.create_index('recipe_url', unique=True)
    for i, recipe_dict in enumerate(recipe_list):
        url = recipe_dict['recipe_url']
        print(f"{coll}: {i} of {len(recipe_list)} => {url}")
        cfTitle = re.sub(r"\s|/|\.|,|'|-|\?|\||<|>|\\|\*|:", '_', recipe_dict['recipe_title'])
        cfTitle = re.sub(r'"', '', cfTitle)
        cfTitle = re.sub(r'__', '_', cfTitle)
        if cfTitle[0:1] == "_":
            cfTitle = cfTitle[1:]
        if cfTitle[-1:] == "_":
            cfTitle = cfTitle[:-1]
        cfTitle = cfTitle[:32]
        print(f"cfTitle: {cfTitle}")
        cacheFilePath = f"/home/shortpoet/Sources/serious_eats_html_caches/{cfTitle}.txt"
        print(f"cfp: {cacheFilePath}")

        if os.path.isfile(cacheFilePath):
            with open(cacheFilePath, encoding='utf-8') as cacheFile:
                html = cacheFile.read()
        else:
            driver.get(url)
            wait = WebDriverWait(driver, 10)
            wait.until(
                EC.presence_of_element_located((By.XPATH, "//script[@type='application/ld+json']")))
            html = driver.page_source          
            with open(cacheFilePath, 'w', encoding='utf-8') as cacheFile:
                cacheFile.write(html)
        se_soup = bs(html, 'html.parser')
        script_divs = se_soup.find_all('script', {'type': 'application/ld+json'})
#             recipe_dict['category_trees'] = []
#             recipe_dict['recipe'] = []
        category_trees = []
        for index, div in enumerate(script_divs):
            try:
                j = json.loads(div.text, strict=False)

                recipe_dict.setdefault('category_trees', [])
                recipe_dict.setdefault('recipe', {})

                if j['@type'] == 'BreadcrumbList':
                    category_tree = []
                    for item in j['itemListElement']:
                        category_tree.append(item['item']['name'])
    #                     recipe_dict['category_trees'].append(category_tree)
                    category_trees.append(category_tree)

                if j['@type'] == 'Recipe':
                    try:
                        rating_count = j['aggregateRating']['ratingCount']
                    except:
                        rating_count = False
                    try:
                        rating_value = j['aggregateRating']['ratingValue']
                    except:
                        rating_value = False
                    try:
                        author = j['author']['name']
                    except:
                        author = False
                    try:
                        job_title = j['author']['jobTitle']
                    except:
                        job_title = False
                    try:
                        description = j['description']
                    except:
                        description = False
                    try:
                        keywords = j['keywords']
                    except:
                        keywords = False
                    try: 
                        name = j['name']
                    except:
                        name = False
                    try:
                        categories = j['recipeCategory']
                    except:
                        categories = False
                    try:
                        cuisine = j['recipeCuisine']
                    except:
                        cuisine = False
                    try:
                        ingredients = j['recipeIngredient']
                    except:
                        ingredients = False
                    try:   
                        steps = {}
                        for i, step in enumerate(j['recipeInstructions']):
                            this = 'step_' + str(i+1)
                            steps[this] = step['text']
                    except:
                        steps = False
                    try:
                        recipe_yield = j['recipeYield']
                    except:
                        recipe_yield = False
                    try:
                        time = str(isodate.parse_duration(j['totalTime']).total_seconds()/60)
                    except:
                        time = False
    #                     recipe_dict['recipe'] = {
                    json_recipe = {
                        'name': name, 'rating_count': rating_count, 'rating_value': rating_value, 'author': author,
                        'job_title': job_title, 'description': description, 'keywords': keywords, 'categories': categories,
                        'cuisine': cuisine, 'ingredients': ingredients, 'steps': steps, 'recipe_yield': recipe_yield, 'time':time
                    }
            except ValueError:
                print('==================')
                print(f"      json error on:{url} - script_div{index}")
                print('==================')
                json_recipe = ''
                continue
        this_recipe = {
            'topic': recipe_dict['topic'], 'source': 'serious_eats', 
            'recipe_title': recipe_dict['recipe_title'], 'recipe_url': recipe_dict['recipe_url'], 
#                 'category_trees': recipe_dict['category_trees'], 'recipe': recipe_dict['recipe'], 
            'category_trees': category_trees, 'recipe': json_recipe, 
            'uploaded': dt.now()
        }
        try:
            collection.insert_one(this_recipe)
        except pymongo.errors.DuplicateKeyError:
            continue
#             collection.update_one({}, {'$set': collection.this_recipe}, upsert=True)
